### StarCoder 2

#### 首先讀取模型權重與分詞器，並將 Generate 包裝成函式方便後續使用：

In [1]:
import torch
from transformers import GPT2Tokenizer as TkCls
from transformers import Starcoder2ForCausalLM as ModelCls
from transformers import TextStreamer

model_id= "bigcode/starcoder2-7b"
model: ModelCls = ModelCls.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
)
tk: TkCls = TkCls.from_pretrained(model_id)
tk.pad_token = tk.eos_token
ts = TextStreamer(tk)

def generate(prompt, n):
    inputs = tk(prompt, return_tensors="pt").to("cuda")
    return model.generate(**inputs, max_new_tokens=n, streamer=ts)

/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.08it/s]


#### 一般的程式碼生成，只需要透過模型本身的文字接龍能力即可達成：

In [8]:
generate("def fib(n: int):", 28)

# # 是個非常標準的費波那契數函式
# """
# def fib(n: int):
#     if n == 0:
#         return 0
#     elif n == 1:
#         return 1
#     else:
#         return fib
# """

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


def fib(n: 

int):
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)




tensor([[  610, 28319,    45,   115,    63,   648,   731,   303,   434,   329,
           350,   244,    55,    63,   310,   461,   329,   303,   461, 28319,
            45,   115,   449,   244,    54,    46,   494, 28319,    45,   115,
           449,   244,    55,    46,   499]], device='cuda:0')

#### [**填充程式碼 (Fill in the Middle, FIM**)](https://arxiv.org/abs/2207.14255)
* 在文字編輯器的游標前後或多或少都有其他程式碼，寫程式時不僅要參考前面的程式碼，也要參考後面的程式碼。然而Decoder 的特性使他只能往後生成文字，而不能在中間進行生成。

* 這時候 StarCoder 透過特殊的 FIM Token 來完成填充程式碼的任務。

* 舉例來說：
    ```python
    def hello (name: str):
        print # <=鍵盤游標在這!

    def goodbye (name: str):
        print(f"### 系統：再會了，{name}!")
    ```
    * 這裡使用 `<游標在這>` 來表示使用者的游標停在此處
    * 並根據游標位置將程式碼切成兩半，前半段是 **Prefix** 後半段是 **Suffix**
    * 將 Prefix 的內容放在 `<fim_prefix>` 後面
    * 將 Suffix 的內容放在 `<fim_suffix>` 後面
    * 最後放上 `<fim_middle>` 來發動 StarCoder 填充程式碼的能力
    ```txt
    <fim_prefix> [前半段的內容] <fim_suffix> [後半段的內容] <fim_middle>
    ```
    
* 將這種格式的提示輸入模型，就能夠產生中間的程式碼。
    * 模型能夠具有這樣的能力，就是因為在訓練的時候，FIM 訓練會將挖空的程式碼片段放在`<fim_middle>` 後面。如此一來，模型就可以在維持原本文字接龍能力的情況下，獲得程式碼填充的能力。
    * StarCoder 2 在 FIM 模式下用來停止生成的 Token 是 `<file_sep>`

In [9]:
def generate(prompt, n):
    inputs = tk(prompt, return_tensors="pt").to("cuda")
    return model.generate(
        **inputs,
        max_new_tokens=n,
        streamer=ts,
        eos_token_id=tk.encode("<file_sep>")[-1],
    )

# 實際操作這種推論方法:

full_code = """

def hello(name: str):
    print(<游標在這>)

def goodbye(name: str):
    print(f"### 系統：再會了，{name}!")

"""

prefix, suffix = full_code.split("<游標在這>", 1)
full_prompt = f"<fim_prefix>{prefix}<fim_suffix>{suffix}<fim_middle>"
outputs = generate(full_prompt, 16)

# # 得到以下輸出:
# """
# <fim_prefix>

# def hello(name: str):
#     print(<fim_suffix>)

# def goodbye(name: str):
#     print(f"### 系統：再會了，{name}!")

# <fim_middle>f"### 系統：你好，{name}!"<file_sep>
# """

Setting `pad_token_id` to `eos_token_id`:6 for open-end generation.


<fim_prefix>

def hello(name: str):
    print(<fim_suffix>)

def goodbye(name: str):
    print(f"### 系統：再會了，{name}!")

<fim_middle>f"### 系統：你好，{name}!"<file_sep>


#### 最後透過一些後處理，將填充程式碼的生成結果重建回原本的程式碼:

In [11]:
tokens = tk.encode(full_prompt)
output = outputs[0][len(tokens):]
middle = tk.decode(output, skip_special_tokens=True)
print(full_code.replace("<游標在這>", middle))

# # 得到最後完整的程式碼如下:
# """
# def hello(name: str):
#     print(f"### 系統：你好，{name}!")

# def goodbye(name: str):
#     print(f"### 系統：再會了，{name}!")
# """



def hello(name: str):
    print(f"### 系統：你好，{name}!")

def goodbye(name: str):
    print(f"### 系統：再會了，{name}!")


